In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd, remove, chdir
from os.path import join, basename

sys.path.append(str(Path(getcwd()).parent))
current_dir = getcwd()
print(current_dir)

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)

import numpy as np
from random import randint
from math import floor, ceil
import codecs

In [ ]:
from bulletin import root, default_input, default_output, agora, hoje, ontem, anteontem, dias_apos, dias_apos_label
from bulletin.services.metabase import Metabase
from bulletin.systems.casos_confirmados import CasosConfirmados
from bulletin.systems.notifica import Notifica
from bulletin.utils.static import Municipios
from bulletin.utils import utils, static
from bulletin.utils.xls_ import fit_cols
from bulletin.utils.normalize import trim_overspace
from bulletin.utils.normalize import normalize_text#normalize_hash, normalize_labels, , date_hash, normalize_number

In [ ]:
from datetime import datetime, date, timedelta
from tqdm import tqdm

exclusao_pathfile = join(root, 'database', 'casos_confirmados')

today = pd.to_datetime(date.today())
ontem = today - timedelta(1)
anteontem = ontem - timedelta(1)
data_retroativos = ontem - timedelta(31)

### 1. Ajuste dos casos do CC

In [ ]:
cc = CasosConfirmados()

date = ontem

cc.load(f"cc_{date.strftime('%d_%m_%Y')}", compress=True)

cc.df.shape

In [ ]:
casos = cc.df.copy()

In [ ]:
cc.df.loc[(cc.df['data_diagnostico'].dt.year == 2022) & (cc.df['data_cura_obito'].dt.year == 2021) & (cc.df['evolucao'] == 2)]

In [ ]:
casos.loc[casos['data_diagnostico'].isna()]

In [ ]:
columns = ['data_coleta', 'data_liberacao']
days_diff = 3

for year in [2020, 2021]:
    
    col = 'data_1o_sintomas'
    casos['diff_days'] = (abs(casos['data_diagnostico'] - casos[col]).dt.days).fillna(0).astype(int)

    #ALTERAÇÃO
    casos.loc[(casos['data_diagnostico'].dt.year==year) 
              & (casos['data_diagnostico'] < casos[col]),'data_diagnostico'] = \
    casos.loc[(casos['data_diagnostico'].dt.year==year) 
              & (casos['data_diagnostico'] < casos[col]), col]
    
    for col in columns:

        casos['diff_days'] = (abs(casos['data_diagnostico'] - casos[col]).dt.days).fillna(0).astype(int)

        #ALTERAÇÃO
        casos.loc[(casos['data_diagnostico'].dt.year==year) 
                  & (casos['data_diagnostico'] < casos[col]) 
                  & (casos['diff_days'] >= days_diff),'data_diagnostico'] = \
        casos.loc[(casos['data_diagnostico'].dt.year==year) 
                  & (casos['data_diagnostico'] < casos[col]) 
                  & (casos['diff_days'] >= days_diff), col]

In [ ]:
casos.loc[casos['data_diagnostico'].isna()]

In [ ]:
casos = casos.drop(columns=['diff_days'])

In [ ]:
# meses = {1:'Janeiro',2:'Fevereiro',3:'Março',4:'Abril',5:'Maio',6:'Junho',7:'Julho',8:'Agosto',9:'Setembro',10:'Outubro',11:'Novembro',12:'Dezembro'}

# casos['ano_caso'] = casos['data_diagnostico'].dt.year
# casos['mes_caso'] = casos['data_diagnostico'].dt.month


# obitos = casos.loc[(casos['evolucao']==2)]
# obitos['ano_obito'] = obitos['data_cura_obito'].dt.year
# obitos['mes_obito'] = obitos['data_cura_obito'].dt.month


# # CASOS E ÓBITOS POR MÊS
# tabela_casos_mes = casos.groupby(['ano_caso', 'mes_caso'])[['id_notifica']].count().rename(columns={'id_notifica':'casos'})
# tabela_casos_mes.rename(index=meses, inplace=True)

# tabela_obitos_mes = obitos.groupby(['ano_obito', 'mes_obito'])[['id_notifica']].count().rename(columns={'id_notifica':'óbitos'})
# tabela_obitos_mes.rename(index=meses, inplace=True)


# tabela_casos_mes.index.set_names(['ano', 'mes'], inplace=True)
# tabela_obitos_mes.index.set_names(['ano', 'mes'], inplace=True)
# tabela_mes = tabela_casos_mes.join(tabela_obitos_mes)
# tabela_mes

In [ ]:
# casos = casos.drop(columns=['ano_caso', 'mes_caso', 'ano_obito', 'mes_obito'])

In [ ]:
casos.shape

In [ ]:
raise

In [ ]:
cc.df = casos
cc.save(f"cc_{date.strftime('%d_%m_%Y')}", replace=True, compress=True)

### 2. Ajuste das fichas no Notifica

In [ ]:
notifica = Notifica()
notifica.load('notifica', compress=False)
notifica.df = notifica.df.drop_duplicates('id', keep='last')
notifica.df.shape

In [ ]:
len(notifica.df.loc[notifica.df['data_diagnostico'].isna()])

In [ ]:
columns = ['data_cadastro','data_coleta','data_recebimento','data_liberacao']
days_diff = 3

for year in [2020, 2021]:
    
    col = 'data_1o_sintomas'
    notifica.df['diff_days'] = (abs(notifica.df['data_diagnostico'] - notifica.df[col]).dt.days).fillna(0).astype(int)

    #ALTERAÇÃO
    notifica.df.loc[(notifica.df['data_diagnostico'].dt.year==year) 
              & (notifica.df['data_diagnostico'] < notifica.df[col]),'data_diagnostico'] = \
    notifica.df.loc[(notifica.df['data_diagnostico'].dt.year==year) 
              & (notifica.df['data_diagnostico'] < notifica.df[col]), col]
    
    for col in columns:

        notifica.df['diff_days'] = (abs(notifica.df['data_diagnostico'] - notifica.df[col]).dt.days).fillna(0).astype(int)

        #ALTERAÇÃO
        notifica.df.loc[(notifica.df['data_diagnostico'].dt.year==year) 
                  & (notifica.df['data_diagnostico'] < notifica.df[col]) 
                  & (notifica.df['diff_days'] >= days_diff),'data_diagnostico'] = \
        notifica.df.loc[(notifica.df['data_diagnostico'].dt.year==year) 
                  & (notifica.df['data_diagnostico'] < notifica.df[col]) 
                  & (notifica.df['diff_days'] >= days_diff), col]

In [ ]:
len(notifica.df.loc[notifica.df['data_diagnostico'].isna()])

In [ ]:
notifica.df = notifica.df.drop(columns=['diff_days'])

In [ ]:
notifica.df.shape

In [ ]:
raise

In [ ]:
notifica.save(replace=True, compress=False)